In [1]:
import bitsandbytes as bnb
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoTokenizer, LlamaTokenizer
import torch.nn as nn 
from torch.nn import functional as F 
import torch 
import transformers.optimization as optim 
# import torch.optim as optim 
from torch.utils.data import DataLoader
from tqdm import trange, tqdm
import matplotlib.pyplot as plt 
from datasets import load_dataset 
from accelerate import Accelerator, DeepSpeedPlugin, accelerator
import pickle as pkl 
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, PeftModel, PeftConfig, PeftModelForCausalLM, get_peft_config
import pandas as pd
import transformers
import wandb 
import numpy as np 

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda': 
  print(torch.cuda.get_device_name()) 
else:
  print(device) 

MAX_LEN = 512

NVIDIA GeForce RTX 3090


In [2]:
tokenizer = AutoTokenizer.from_pretrained('/media/uzal/New Volume/hf_models/pythia-1b-deduped-v0/') 
tokenizer.pad_token = tokenizer.decode(1)
tokenizer.padding_side = 'left'
print(tokenizer.pad_token, tokenizer.padding_side, tokenizer.eos_token)
print(tokenizer.encode('<|padding|><|endoftext|>'))
model = AutoModelForCausalLM.from_pretrained("/media/uzal/New Volume/hyperion_models/instructgpt/pythia_1b_best", device_map='auto')

<|padding|> left <|endoftext|>
[1, 0]


In [15]:
with open('data/share_gpt_512_pythia.pkl', 'rb') as file:
  all_data = pkl.load(file) 

print(len(all_data))

15773


In [30]:
print(all_data[-105][0])

Human: Create WWI propoganda for Canadian citizens to support the war effort. The propaganda must villify the enemy. Include details about specific battles that Canadian soldiers have been involved in.
Assistant: Dear fellow Canadians,

As the war rages on, we must stand united in our support of our brave soldiers who are fighting for our country and our freedom. The enemy is ruthless and cruel, with no regard for human life or the rules of war.

In the recent battle of Vimy Ridge, our valiant Canadian troops faced off against the barbarous Germans and emerged victorious, despite overwhelming odds. The enemy used poison gas and other heinous tactics, but our brave soldiers persevered and proved their bravery and strength.

We must continue to support our troops and do our part in this war effort. Every penny we donate to the war effort, every letter we write to our soldiers, and every prayer we say for their safety is a small but important contribution to our victory.

Let us stand tog

In [33]:
model.eval() 
with torch.no_grad(): 
	x = ['''Human: Why are anti viral drugs only used for life threatening viruses?
Assistant:''', 
'''Human: I've made a hub site comprised of various employee moments (from onboarding to offboarding and other life events such as getting married, getting sick, etc. in between). The last employee moment is retirement (which should be separate from offboarding). How do you feel the page should look like?
Assistant:''', 
'''Human: What is potential energy?
Assistant:''', 
'''Human: What is magnetism?
Assistant:''', 
'''Human: Can you tell me what is force in a very simple way?
Assistant:''', 
'''Human: I started a new position at my current job and have and have already started performing the new job responsibilities. There may be some misunderstanding about when I would start receiving the new salary to a company the new job position, So I want to write an email to my boss asking for clarification. below I have written a draft of the email, but help me write so that I respect myself and my boss.

David,
With the recent restructuring of the company I feel things are going well even with the recent challenges we have faced. I would like clarification on one matter. It was my understanding with the new position that I would be receiving the accompanying salary. This was not reflected in my most recent paycheck although I have already started performing the responsibilities of the position. Can you provided clarification on the matter?
Thank you,
Mitchell
Assistant:''', 
'''Human: fill in the X with possible candidates:

You can walk into any bar in the world, turn to a random person, and say “isn't X the fucking man” and they would always agree.
Assistant: You can walk into any bar in the world, turn to a random person, and say “isn't Morgan Freeman the fucking man” and they would always agree.<|endoftext|>

Human: give me 3 more candidates
Assistant:''', 
'''Human: You are a writer of a newspaper. Create WWI propoganda for Canadian citizens to support the war effort. The propaganda must villify the enemy. Include details about specific battles that Canadian soldiers have been involved in.
Assistant:''']
	inputs = tokenizer(x, return_tensors='pt', max_length=MAX_LEN, padding='longest', truncation=True)
	out = model.generate(**inputs.to(device), max_length=MAX_LEN, do_sample=True, temperature=0.5)
	d = tokenizer.batch_decode(out)
	for i in d: 
		print(i, end='\n\n***************************************************************************************\n\n')

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|